In [18]:
p = 2
m = 4
irreducible_poly = [1,1,0,1]

In [19]:
def generate_field_elements(comb, sol, idx):
  if(idx == m):
    sol.add(tuple(comb))
    return
  for i in range(p):
    comb.append(i)
    generate_field_elements(comb, sol, idx + 1)
    comb.pop()
  return sol

In [20]:
sol = generate_field_elements([], set(), 0)
field_elements = list(map(list, sol))

for i in range(p ** m):
  while(field_elements[i] and field_elements[i][-1] == 0): field_elements[i].pop()
  if(field_elements[i] == []): field_elements[i] = [0]

In [21]:
# list(map(int, '1 3 3 4 2'.split()))[::-1]

In [22]:
field_elements

[[0, 0, 0, 1],
 [0, 0, 1],
 [0, 1, 0, 1],
 [0, 1, 1, 1],
 [1, 0, 1, 1],
 [1, 1],
 [0, 1],
 [1, 1, 1],
 [0, 1, 1],
 [0],
 [1, 0, 1],
 [1, 0, 0, 1],
 [1, 1, 0, 1],
 [1],
 [0, 0, 1, 1],
 [1, 1, 1, 1]]

In [23]:
def generate_polynomial(poly):
    ans = ""
    for i in range(len(poly)):
        if(poly[i] == 0): continue
        else: ans += f"{poly[i]} x**{i} + "
    if(ans): ans = ans[:-2]
    else: return '0'
    return ans.strip()

field_elements_polynomials = {}
for ele in field_elements:
  field_elements_polynomials[generate_polynomial(ele)] = ele

field_elements_polynomials

{'1 x**3': [0, 0, 0, 1],
 '1 x**2': [0, 0, 1],
 '1 x**1 + 1 x**3': [0, 1, 0, 1],
 '1 x**1 + 1 x**2 + 1 x**3': [0, 1, 1, 1],
 '1 x**0 + 1 x**2 + 1 x**3': [1, 0, 1, 1],
 '1 x**0 + 1 x**1': [1, 1],
 '1 x**1': [0, 1],
 '1 x**0 + 1 x**1 + 1 x**2': [1, 1, 1],
 '1 x**1 + 1 x**2': [0, 1, 1],
 '0': [0],
 '1 x**0 + 1 x**2': [1, 0, 1],
 '1 x**0 + 1 x**3': [1, 0, 0, 1],
 '1 x**0 + 1 x**1 + 1 x**3': [1, 1, 0, 1],
 '1 x**0': [1],
 '1 x**2 + 1 x**3': [0, 0, 1, 1],
 '1 x**0 + 1 x**1 + 1 x**2 + 1 x**3': [1, 1, 1, 1]}

In [24]:
def degree(poly):
    while poly and poly[-1] == 0:
        poly.pop()   # normalize
    return len(poly)-1

def congruence(n,m):
    while ((n % p) % m) != 0:
        n = n + p
    return n

def divide(u, v):
    q, r = [0], u
    divisorDeg = degree(v)
    divisorLC = v[-1]

    while degree(r) >= divisorDeg:
        monomialExponent = degree(r) - divisorDeg
        monomialZeros = [0 for _ in range(monomialExponent)]
        if r[-1] % divisorLC != 0:
            monomialDivisor = monomialZeros + [congruence(r[-1], divisorLC) // divisorLC]
        else:
            monomialDivisor = monomialZeros + [r[-1] // divisorLC]

        q = add(q, monomialDivisor)
        r = subtract(r, multiply(monomialDivisor, v))

    return q, r

In [25]:
def rem(u, v):
    n, m = degree(u), degree(v)
    if m < 0: raise ZeroDivisionError
    if n >= m:
        while n >= m:
            d = [0] * (n - m) + v
            if u[-1] % float(v[-1]) != 0:
                mult = congruence(u[-1],float(v[-1])) / float(v[-1])
            else:
                mult = u[-1] / float(d[-1])
            d = [(coeff * mult) % p for coeff in d]
            u = [(coeffu - coeffd) % p for coeffu, coeffd in zip(u, d)]
            n = degree(u)
        r = u
    else:
        r = u
    return r

In [26]:
def add(u, v):
  n, m = len(u), len(v)
  m_, ans = max(n, m), []
  u = u + [0] * (m_ - n)
  v = v + [0] * (m_ - m)
  for i in range(m_):
    ans.append((u[i] + v[i]) % p)
  while(ans and ans[-1] == 0):
    ans.pop()
  if(not ans): return [0]
  r = rem(ans, irreducible_poly)
  return r

In [27]:
#  u(x) - v(x)
def subtract(u, v):
  n, m = len(u), len(v)
  m_, ans = max(n, m), []
  u = u + [0] * (m_ - n)
  v = v + [0] * (m_ - m)
  for i in range(m_):
    ans.append((u[i] - v[i]) % p)
  while(ans and ans[-1] == 0):
    ans.pop()
  if(not ans): return [0]
  r = rem(ans, irreducible_poly)
  return r

In [28]:
def multiply(u, v):
  n, m = len(u), len(v)
  ans = [0 for _ in range(n + m)]
  for i in range(n):
    for j in range(m):
      ans[i + j] += u[i] * v[j]
  for i in range(n + m):
    ans[i] %= p
  while(ans and ans[-1] == 0):
    ans.pop()
  if(not ans): return [0]
  r = rem(ans, irreducible_poly)
  return r

In [29]:
def find_inverse():
  d = {}
  for ele in field_elements:
    for ele2 in field_elements:
      if(tuple(ele2) in d): continue
      if(multiply(ele, ele2) == [1]):
        d[tuple(ele)] = tuple(ele2)
        d[tuple(ele2)] = tuple(ele)
  return d

In [30]:
inverse = find_inverse()

In [31]:
inverse

{(0, 0, 0, 1): (0, 1, 1),
 (1, 0, 1, 1): (1, 1),
 (0, 1, 1): (0, 0, 0, 1),
 (0, 0, 1): (0, 0, 1, 1),
 (1, 1, 1): (1, 1, 1, 1),
 (0, 0, 1, 1): (0, 0, 1),
 (0, 1, 0, 1): (1,),
 (1,): (0, 1, 0, 1),
 (0, 1, 1, 1): (1, 0, 0, 1),
 (0, 1): (1, 0, 1),
 (1, 0, 0, 1): (0, 1, 1, 1),
 (1, 1): (1, 0, 1, 1),
 (1, 0, 1): (0, 1),
 (1, 1, 1, 1): (1, 1, 1)}

# Parsing

In [32]:
# f1 + f2 - f3 + (f4 + f5 * f8) / f6 * f7

def operandStack(listNumbers: str):
    numbers = listNumbers[1:len(listNumbers) - 1].split(',')
    return list(map(int, numbers))

def splitInputString(input: str) -> list:
    input = input.replace(' ', '')
    inputStack, inputOperand, index = [], [], 0
    # Check for balanced parentheses, otherwise throw error
    parenthesesArray = [char for char in input if char in ['(', ')']]
    balancedStack = []
    for element in parenthesesArray:
        if element == '(':
            balancedStack.append(element)
        else:
            if len(balancedStack) == 0:
                print("Entered expression is incorrect, re-enter the expression.")
                return
            else:
                balancedStack.pop()
    if len(balancedStack) != 0:
        print("Entered expression is incorrect, re-enter the expression.")
        return

    # Checking for balanced polynomial brackets
    bracketsArray = [char for char in input if char in ['[', ']']]
    balancedStack = []
    for element in bracketsArray:
        if element == '[':
            balancedStack.append(element)
            if len(balancedStack) > 1:
                print("Entered expression is incorrect, re-enter the expression.")
                return
        else:
            if len(balancedStack) == 0:
                print("Entered expression is incorrect, re-enter the expression.")
                return
            else:
                balancedStack.pop()
    if len(balancedStack) != 0:
        print("Entered expression is incorrect, re-enter the expression.")
        return

    while index < len(input):
        if input[index] in ['+', '-', '*', '/', '(', ')']:
            inputStack.append(input[index])
            index += 1
        else:
            if input[index] == '[':
                inputOperand = []
                closingBracket = index + input[index:].index(']')
                inputStack.append(operandStack(input[index:closingBracket + 1]))
                index = closingBracket + 1

    # Two consecutive operators or two consecutive operands cannot be evaluated together (for example: a * + b, a b * c)
    for index in range(0, len(inputStack) - 1, 1):
        if inputStack[index] in ['+', '-', '*', '/'] and inputStack[index + 1] in ['+', '-', '*', '/']:
            print("Entered expression is incorrect, re-enter the expression.")
            return
        if isinstance(inputStack[index], list) and isinstance(inputStack[index + 1], list):
            print("Entered expression is incorrect, re-enter the expression.")
            return
    return inputStack

In [33]:
def parse(inputString: str):
    tokenStack = ['('] + splitInputString(inputString) + [')']
    positionIndex = 0

    def tokenStackElement():
        nonlocal positionIndex
        return tokenStack[positionIndex]

    def moveIndex():
        nonlocal positionIndex
        positionIndex += 1

    def parsePrimaryExpression():
        token = tokenStackElement()
        # If token type is a polynomial
        if isinstance(token, list) and positionIndex < len(tokenStack):
            moveIndex()
            return token
        elif token == '(' and positionIndex < len(tokenStack):
            moveIndex()
            expression = parseExpression()
            moveIndex()
            return expression

    def parseMulExpression():
        expression = parsePrimaryExpression()
        token = tokenStackElement()
        while (positionIndex < len(tokenStack) and (token == '*' or token == '/')):
            moveIndex()
            rightHandExpression = parsePrimaryExpression()
            expression = (token, expression, rightHandExpression)
            token = tokenStackElement()
        return expression

    def parseExpression():
        expression = parseMulExpression()
        token = tokenStackElement()
        while (positionIndex < len(tokenStack) and (token == '+' or token == '-')):
            moveIndex()
            rightHandExpression = parseMulExpression()
            expression = (token, expression, rightHandExpression)
            token = tokenStackElement()
        return expression

    result = parsePrimaryExpression()

    return result

In [34]:
def evaluateExpr(expressionTuple):
    if isinstance(expressionTuple, list):
        return expressionTuple
    elif isinstance(expressionTuple, tuple):
        operator, operator1, operator2 = expressionTuple
        if operator == '+':
            return add(evaluateExpr(operator1), evaluateExpr(operator2))
        elif operator == '-':
            return subtract(evaluateExpr(operator1), evaluateExpr(operator2))
        elif operator == '*':
            return multiply(evaluateExpr(operator1), evaluateExpr(operator2))
        elif operator == '/':
            return multiply(evaluateExpr(operator1), inverse[tuple(evaluateExpr(operator2))])

In [35]:
parse('[1, 2, 3] * [4, 5] + ([6, 7] / [7, 8] + [9, 10])')

('+', ('*', [1, 2, 3], [4, 5]), ('+', ('/', [6, 7], [7, 8]), [9, 10]))

In [36]:
parse('[1, 2, 3] + ([4, 5] * [5, 7] - [8, 9])')

('+', [1, 2, 3], ('-', ('*', [4, 5], [5, 7]), [8, 9]))

In [37]:
e = parse('([1, 1, 1] * [1, 1]) + [1, 0, 1] / [0, 1, 1]')

In [38]:
evaluateExpr(e)

[0.0, 1.0, 1.0]

In [39]:
inverse[tuple([0,1,1])]

(0, 0, 0, 1)

In [40]:
multiply([1,1,1],[1,1])

[0.0, 1.0]